In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime
import scipy.stats as stats

### Read data and show some statistics

In [18]:
solar_df = pd.read_csv("hessi.solar.flare.UP_To_2018.csv", sep=',', index_col=0)

''' Drop the flag columns'''
solar_df = solar_df.drop(["active.region.ar", "flag.1", "flag.2", "flag.3", "flag.4", "flag.5"], axis=1)

''' Some Radial Values are beyond 2000'''
solar_df = solar_df[solar_df["radial"] <960]

In [19]:
len(solar_df)

94559

In [20]:
solar_df.head(5)

,start.date,start.time,peak,end,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial
flare,,,,,,,,,,,
2021213,2002-02-12,21:29:56,21:33:38,21:41:48,712,136,167304.0,12-25,592,-358,692
2021228,2002-02-12,21:44:08,21:45:06,21:48:56,288,7,9504.0,6-12,604,-341,694
2021332,2002-02-13,00:53:24,00:54:54,00:57:00,216,15,11448.0,6-12,-310,375,487
2021308,2002-02-13,04:22:52,04:23:50,04:26:56,244,20,17400.0,12-25,-277,378,469
2021310,2002-02-13,07:03:52,07:05:14,07:07:48,236,336,313392.0,25-50,-272,390,476


In [21]:
solar_df.isnull().sum()

start.date      0
start.time      0
peak            0
end             0
duration.s      0
peak.c/s        0
total.counts    0
energy.kev      0
x.pos.asec      0
y.pos.asec      0
radial          0
dtype: int64

In [22]:
solar_df.describe()

,duration.s,peak.c/s,total.counts,x.pos.asec,y.pos.asec,radial
count,94559.000000,94559.000000,9.455900e+04,94559.000000,94559.000000,94559.000000
mean,484.351484,219.663342,3.687067e+05,-1.492846,-34.227879,604.940302
std,426.727985,885.057346,3.148810e+06,620.340826,251.389632,288.761519
min,8.000000,0.000000,8.000000e+00,-959.000000,-959.000000,0.000000
25%,208.000000,28.000000,2.226600e+04,-594.000000,-246.000000,405.000000
50%,360.000000,56.000000,5.755200e+04,0.000000,-62.000000,657.000000
75%,616.000000,144.000000,1.767480e+05,597.000000,190.000000,866.000000
max,4444.000000,113156.000000,4.355501e+08,959.000000,957.000000,959.000000


### Data Preprocessing

In [23]:
# parse date, time
def parse_date(sdatex, stimex):
        datex = datetime.strptime(sdatex, '%Y-%m-%d')
        timex = datetime.strptime(stimex, '%H:%M:%S')
        return datetime(datex.year,datex.month,datex.day,timex.hour,timex.minute,timex.second)

In [24]:
pd_df = solar_df.copy(deep=True)

In [25]:
# Adding year, month, day, start date, peak date, end date and dropping earlier columns
pd_df['dt.start'] = pd_df[['start.date','start.time']].apply(lambda x: parse_date(x[0],x[1]), axis=1)
pd_df['dt.peak'] = pd_df[['start.date','peak']].apply(lambda x: parse_date(x[0],x[1]), axis=1)
pd_df['dt.end'] = pd_df[['start.date','end']].apply(lambda x: parse_date(x[0],x[1]), axis=1)

pd_df.drop(['start.date','start.time','peak','end'], axis=1, inplace=True)

# add new columns
pd_df['year'] = pd_df['dt.start'].apply(lambda col: col.year)
pd_df['month'] = pd_df['dt.start'].apply(lambda col: col.month)
pd_df['day'] = pd_df['dt.start'].apply(lambda col: col.day)

In [26]:
pd_df = pd_df.rename(columns={'duration.s': 'duration', 'peak.c/s': 'peak_c_s', 'total.counts': 'total_counts', 
                                    'energy.kev': 'energy_kev', 'x.pos.asec': 'x_pos', 'y.pos.asec': 'y_pos', 
                                    'dt.start': 'date_start', 'dt.peak':'date_peak', 'dt.end': 'date_end'})

In [27]:
pd_df.head(5)

,duration,peak_c_s,total_counts,energy_kev,x_pos,y_pos,radial,date_start,date_peak,date_end,year,month,day
flare,,,,,,,,,,,,,
2021213,712,136,167304.0,12-25,592,-358,692,2002-02-12 21:29:56,2002-02-12 21:33:38,2002-02-12 21:41:48,2002,2,12
2021228,288,7,9504.0,6-12,604,-341,694,2002-02-12 21:44:08,2002-02-12 21:45:06,2002-02-12 21:48:56,2002,2,12
2021332,216,15,11448.0,6-12,-310,375,487,2002-02-13 00:53:24,2002-02-13 00:54:54,2002-02-13 00:57:00,2002,2,13
2021308,244,20,17400.0,12-25,-277,378,469,2002-02-13 04:22:52,2002-02-13 04:23:50,2002-02-13 04:26:56,2002,2,13
2021310,236,336,313392.0,25-50,-272,390,476,2002-02-13 07:03:52,2002-02-13 07:05:14,2002-02-13 07:07:48,2002,2,13


In [28]:
# Enumerating energy range values from str to category
category = {'3-6': 0, '6-12': 1, '12-25': 2, '25-50': 3, '50-100': 4, '100-300': 5, '300-800': 6, '800-7000': 7, '7000-20000': 8}
pd_df['energy_kev'] = pd_df['energy_kev'].map(category)

In [29]:
# Deal with the skewed duration data using log transform
pd_df['duration'] = np.log1p(pd_df['duration'])

In [30]:
pd_df = pd_df.drop(['date_start', 'date_peak', 'date_end'], axis=1)

### Create Train/Test Set

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
X_train, X_test = train_test_split(pd_df, test_size=0.2)

In [34]:
X_train.shape, X_test.shape

((75647, 10), (18912, 10))

### Predcition for energy_kev
Best result - Random Forest 0.87

In [35]:
y_train_energy = X_train['energy_kev']
X_train_energy = X_train.drop(['energy_kev', 'duration'], axis=1)

y_test_energy = X_test['energy_kev']
X_test_energy = X_test.drop(['energy_kev', 'duration'], axis=1)

In [38]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
random_forest_classifier = RandomForestClassifier(n_jobs=-1).fit(X_train_energy, y_train_energy)
random_forest_score = random_forest_classifier.score(X_test_energy, y_test_energy)
random_forest_score_train = random_forest_classifier.score(X_train_energy, y_train_energy)
random_forest_score, random_forest_score_train

(0.8686548223350253, 0.9999735614102344)

### Prediction for duration
Best result - Random Forest 0.8496

In [37]:
y_train_duration = X_train['duration']
X_train_duration = X_train.drop(['energy_kev', 'duration'], axis=1)

y_test_duration = X_test['duration']
X_test_duration = X_test.drop(['energy_kev', 'duration'], axis=1)

In [43]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor
random_forest_regressor = RandomForestRegressor(n_jobs=-1).fit(X_train_duration, y_train_duration)
random_forest_predictions = random_forest_regressor.predict(X_train_duration)
random_forest_predictions_test = random_forest_regressor.predict(X_test_duration)
random_forest_MSE_train = mean_squared_error(y_train_duration, random_forest_predictions)
random_forest_MSE = mean_squared_error(y_test_duration, random_forest_predictions_test)

random_forest_score = random_forest_regressor.score(X_test_duration, y_test_duration)
random_forest_score_train = random_forest_regressor.score(X_train_duration, y_train_duration)
random_forest_score, random_forest_score_train, random_forest_MSE, random_forest_MSE_train

(0.8455543644863435,
 0.9782757767844703,
 0.11010360733002275,
 0.015161059255677307)

### Prediction for x.pos
Best result - Random Forest 0.69

In [44]:
y_train_xpos = X_train['x_pos']
X_train_xpos = X_train.drop(['x_pos', 'y_pos'], axis=1)

y_test_xpos = X_test['x_pos']
X_test_xpos = X_test.drop(['x_pos', 'y_pos'], axis=1)

In [45]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor
random_forest_regressor = RandomForestRegressor(n_jobs=-1).fit(X_train_xpos, y_train_xpos)
random_forest_predictions = random_forest_regressor.predict(X_test_xpos)
random_forest_score = random_forest_regressor.score(X_test_xpos, y_test_xpos)
random_forest_score_train = random_forest_regressor.score(X_train_xpos, y_train_xpos)
random_forest_score, random_forest_score_train

(0.6960875247464864, 0.9569849950879823)

### Prediction for y.pos
Best result - Random Forest 0.6355

In [46]:
y_train_ypos = X_train['y_pos']
X_train_ypos = X_train.drop(['x_pos', 'y_pos'], axis=1)

y_test_ypos = X_test['y_pos']
X_test_ypos = X_test.drop(['x_pos', 'y_pos'], axis=1)

In [47]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor
random_forest_regressor = RandomForestRegressor(n_jobs=-1).fit(X_train_ypos, y_train_ypos)
random_forest_predictions = random_forest_regressor.predict(X_test_ypos)
random_forest_score = random_forest_regressor.score(X_test_ypos, y_test_ypos)
random_forest_score_train = random_forest_regressor.score(X_train_ypos, y_train_ypos)
random_forest_score, random_forest_score_train

(0.6731553692155867, 0.9536577720586823)